In [ ]:
from bs4 import BeautifulSoup

In [ ]:
html = '''
<html>
    <head> </head>
    <body> 
        <div id = "result1">
            <p class = row>
                <a class = "red" href = "/link1"> '링크1' </a>
                <a class = "blue" href = "/link2"> '링크2' </a>
            <a href = '/link1'>
            </p>
            </div>
        <div id = "result2">
            <p class = row>
                <a class = "red" href = "/link1"> '링크1' </a>
                <a class = "blue" href = "/link2"> '링크2'</a>
            <a href = '/link1'>
            </p>
            </div>
        </body>

    </html>
'''

dom = BeautifulSoup(html, 'html.parser')

In [ ]:
# p의 자식들 중에 a를 찾아라라
div = dom.find(attrs={'id':'result1'}).find(recursive=False).find_all(recursive=False)

dom.find_all('a')


In [ ]:
dom.select_one('#')

In [ ]:
dom.select('#result1 > p > a')

In [ ]:
dom.select('#result1 > p:first-child > a')

In [ ]:
# CSS의 표현
dom.select('#result1 > p:nth-child(1) > a')

In [ ]:
# "+" : 옆의 자매
dom.select('#result1 > p:has(+ p) > a')


In [ ]:
# 두번째 p를 불러오는 방법법
## 1. p + p
dom.select('#result1 > p + p ') # 두번째 p를 불러 올 수 있다.


In [ ]:
dom.select('#result1 > p')

In [ ]:
# 없는 것 찾기
 오류 생성
dom.find('article').find_all('a') 

In [ ]:
dom.select('article a') 

In [ ]:
from requests import request
from requests.exceptions import HTTPError
from time import sleep
def download(url, params={}, method='GET', retries=3):
    resp = None
    
    try:
        resp = request(method, url,
                       params=params if method=='GET' else {},
                       data=params if method=='POST' else {},
                       headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
        resp.raise_for_status()
        # 아니면,
        # if resp.status_code != 200:
    except HTTPError as e:
        if 500 <= e.response.status_code:
            if retries > 0:
                sleep(3)
                resp = download(url, params=params,
                                method=method,
                                retries=retries-1)
            else:
                print('재방문 횟수 초과')
        else:
            print('Request', resp.request.headers)
            print('Response', e.response.headers)
        
    return resp

In [ ]:
resp = download('http://pythonscraping.com/pages/page3.html')
dom = BeautifulSoup(resp.text, 'html5lib')

In [ ]:
resp.text

In [ ]:
# 

table = dom.find('table') # tbody에 내용이 없다. 
tbody = table.find('tbody') 
for tr in tbody.find_all('tr'):
    print(tr.find_all(recursive = False)[-1])
    print("___________")

In [ ]:
# selector로 찾기 : 한번의 구문으로 img 태그가 달려있는 값까지 접근할 수 있다. 
dom.select('table > tbody > tr > td > img')

In [ ]:
# Selctor와 HTML 구문을 섞어서 쓸 수 있따 : BS의 장점(selector는 부모로 갈 수 없다.)
dom.select('table > tbody > tr > td > img')[0].find_parent()

In [ ]:
dom.select('tr:has(td:has(>img[src$=jpg]))') # 굳이 selctor로 부모를 찾는다면 : has를 활용해야 한다.

In [ ]:
dom.select('tbody > tr')

In [ ]:
# 홀수/짝수 번째 행 찾기
dom.select('tbody > tr:nth-child(2n)') # 짝수 번째 행만 출력하기 
dom.select('tbody > tr:nth-child(even)')

In [ ]:
resp = download('https://www.google.com/search', {'q':'카리나'})


In [ ]:
dom = BeautifulSoup(resp.text, 'html5lib')


In [ ]:
import re
dom.find(text=re.compile('나무위키')).find_parent().find_parent().find_parent().find_parent() # URL 찾아가기 

In [ ]:
# 나무위키 위에 적힌 h3 찾아가기
dom.select('h3') # h3 직접 찾기
dom.select('.LC20lb.MBeuO.DKV0Md') # class 값으로 찾기
dom.select('.LC20lb')
dom.select('.LC20lb.MBeuO')
dom.select('h3.LC20lb')


In [ ]:
dom.select('a[jsname = "UWckNb"]')
# h3
dom.select('a[jsname = "UWckNb"] > h3')
dom.select('a:has(> h3.LC20lb)') # has 뒤에 괄호를 붙여서 써야한다
dom.select('a:has( > .LC20lb.MBeuO.DKV0Md)')

In [ ]:
# 링크와 URL 한번에 가져오기 
for a in dom.select('a:has( > .LC20lb.MBeuO.DKV0Md)'):
    print(a.attrs['href'])
    print(a.select_one('h3').text)

### 예제2 : 다음에서

In [ ]:
resp = download('https://search.daum.net/search', {'w': 'tot', 'q' : '카리나'})
dom = BeautifulSoup(resp.text, 'html5lib')

In [ ]:
# 뉴스의 링크 찾기

# 1) 속성만 사용
dom.select(".tit_main.fn_tit_u") # class 값들 간의 빈칸은 .으로 대체
for a in dom.select(".tit_main.fn_tit_u"):
    print(a.attrs['href'])

In [ ]:
# 2) 
dom.select('.wrap_cont')




In [ ]:
# 3)
dom.select('.wrap_thumb[id] + .wrap_cont')

In [ ]:
dom.select('#container') # id는 반드시 1개
dom.select('#container > li > div:last-child') # div 중에서 마지막 자식
dom.select('#container > li > div:last-child > a') # 그중에서 a
for a in dom.select('#container > li > div:last-child > a') :
    print(a.text)
    print(a.attrs['href'])

In [ ]:
# 위키백과 찾기 : 고유한 값이 없을 경우 ?

dom.select('tit-g.clamp-g') # 안나온다 -> 브라우저에서 보이는 것과 dom에서 다른 경우 (자바스크립트 해석과정에서 달라질 수 있음. )
for title in dom.select('c-doc-web > c-title'):
        print(title.text)
        print(title.attrs['data-href'])


In [ ]:
for a in dom.select('#container > li > div:last-child > a ') :
    print(a.text)
    print(a.attrs[
        
    ])

In [ ]:
dom.select('#container > li > div:last-child > a>')

## Crawling

In [ ]:
URLs = []
while URLs : # URL 리스트에 더 이상 url이 남아있지 않을 때 까지 -> while, 종료 조건
    

In [130]:
import math
math.factorial(10)
math.log10(math.factorial(361))

768.1577357086254

In [129]:
math.factorial(10)

3628800

In [131]:
import requests
from bs4 import BeautifulSoup

url = 'https://news.naver.com/'

response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.content, 'html.parser')

urls = []

for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.startswith('/'):
        # 상대 경로를 절대 경로로 변환
        href = 'https://news.naver.com' + href
    urls.append(href)

# 중복 URL 제거
unique_urls = list(set(urls))

for u in unique_urls:
    print(u)

https://n.news.naver.com/article/422/0000619000
https://n.news.naver.com/article/002/0002300744
https://media.naver.com/press/031
https://n.news.naver.com/article/006/0000118753
https://n.news.naver.com/article/030/0003135982
https://media.naver.com/j/74422?oid=008
https://media.naver.com/press/016#SpecialReport
https://media.naver.com/press/028#SpecialReport
https://n.news.naver.com/article/020/0003520732
https://media.naver.com/press/586
https://news.naver.com/hotissue/main?sid1=110&cid=2000782
https://media.naver.com/press/094
https://n.news.naver.com/article/421/0007051586
https://n.news.naver.com/article/119/0002749325
https://media.naver.com/j/74986?oid=119
https://n.news.naver.com/article/659/0000014904
https://n.news.naver.com/article/421/0007048868
https://n.news.naver.com/article/308/0000033293
https://n.news.naver.com/article/009/0005186858
https://media.naver.com/press/262#SpecialReport
https://n.news.naver.com/article/119/0002749436?sid=104&type=journalists
https://n.news.

## Crawling

In [150]:
from requests.compat import urljoin

url = 'https://www.google.com/search?q=%EC%B9%B4%EB%A6%AC%EB%82%98&oq=%EC%B9%B4%EB%A6%AC%EB%82%98&aqs=chrome.0.69i59j69i61l2.929j0j4&sourceid=chrome&ie=UTF-8'
URLs = []
seens = []
URLs.append(url)

while URLs :
    seed = URLs.pop(0) # 인출하면 제거
    seens.append(seed)
    # robot.txt 확인 -> 원래는 확인해야 한다.
    resp = download(seed)

    if resp.status_code != 200 :
        continue # 200이 아니면 다음 코드로 넘어가기
    
    # text.html 확인
    if re.search('text.html', resp.headers['content-type']) :
        dom = BeautifulSoup(resp.text, 'html5lib')
        # dom에서 3가지를 기준으로 link 후보 가져오기, 정규표현식 활용
        for link in dom.select('*[href], *[src], *[action]') :
            # 걸러서, href 지정해주기
            if link.has_attr('src'):
                href = link.attrs['src']
            elif link.has_attr('href'):
                href = link.attrs['href']
            elif link.has_attr('action'):
                href = link.attrs['action']
            # href는 상대주소, 절대 주소 + 자바스크립트 (#, data, matilto, tel) -> 걸러줘야 한다. 
            # 상대주소이면 붙여주기
            if not re.match('(?:#)|(?:javascript)|(?:data)|(?:mailto)', href):
                newurl = urljoin(resp.request.url, href)
                if newurl not in seens and\
                    newurl not in URLs:
                    URLs.append(newurl)




KeyboardInterrupt: 

In [151]:
len(URLs), len(seens) # 계속 돌아간다. => Focused Crawling의 필요성.

(1801, 21)

1101

### Focused Crawling

In [155]:
from requests.compat import urljoin

url = 'https://www.google.com/search?q=%EC%B9%B4%EB%A6%AC%EB%82%98&oq=%EC%B9%B4%EB%A6%AC%EB%82%98&aqs=chrome.0.69i59j69i61l2.929j0j4&sourceid=chrome&ie=UTF-8'
URLs = []
seens = []
URLs.append((url, 0)) # Tuple로 전환

while URLs :
    seed = URLs.pop(0) # 인출하면 제거
    seens.append(seed[0])
    # robot.txt 확인 -> 원래는 확인해야 한다.
    resp = download(seed[0])

    if resp.status_code != 200 :
        continue # 200이 아니면 다음 코드로 넘어가기
    
    if re.search('text.html', resp.headers['content-type']) :
        dom = BeautifulSoup(resp.text, 'html5lib')
        for link in dom.select('*[href], *[src], *[action]') :
            # 걸러서, href 지정해주기
            if link.has_attr('src'):
                href = link.attrs['src']
            elif link.has_attr('href'):
                href = link.attrs['href']
            elif link.has_attr('action'):
                href = link.attrs['action']

            if not re.match('(?:#)|(?:javascript)|(?:data)|(?:mailto)', href):
                newurl = urljoin(resp.request.url, href)
                # 깊이 제한하기 : 깊이가 3 넘어가면 제한하기 
                if seed[1] >3 :
                    continue

                # 조건문에서 제한하기 
                if newurl not in list(map(lambda r : r[0], seens)) and\
                    newurl not in list(map(lambda r : r[0], URLs)):
                    URLs.append((newurl, seed[1] +1))




KeyboardInterrupt: 

In [156]:
len(URLs), len(seens)

(1178, 16)

In [165]:
URLs[0, :]

TypeError: list indices must be integers or slices, not tuple

In [171]:
len([url for url in URLs if url[1] == 1])

0

### 도메인에 따라 거르기 
- 원하는 도메인의 URL만 구하기 -> ex) google.com으로
- Page Rank의 아이디어 : 어떤 페이지가 더 중요할 까? 

In [172]:
from requests.compat import urljoin, urlparse

url = 'https://www.google.com/search?q=%EC%B9%B4%EB%A6%AC%EB%82%98&oq=%EC%B9%B4%EB%A6%AC%EB%82%98&aqs=chrome.0.69i59j69i61l2.929j0j4&sourceid=chrome&ie=UTF-8'
URLs = []
seens = []
domain = 'https://www.google.com' # 도메인 지정
URLs.append((url, 0))

while URLs :
    seed = URLs.pop(0) 
    seens.append(seed[0])
    resp = download(seed[0])

    if resp.status_code != 200 :
        continue 
    
    if re.search('text.html', resp.headers['content-type']) :
        dom = BeautifulSoup(resp.text, 'html5lib')
        for link in dom.select('*[href], *[src], *[action]') :
            if link.has_attr('src'):
                href = link.attrs['src']
            elif link.has_attr('href'):
                href = link.attrs['href']
            elif link.has_attr('action'):
                href = link.attrs['action']

            if not re.match('(?:#)|(?:javascript)|(?:data)|(?:mailto)', href):
                newurl = urljoin(resp.request.url, href)
                
                if seed[1] >3 :
                    continue

                # 도메인을 기준으로 거르기 : netloc
                # white list, black list
                if urlparse(newurl).netloc not in domain:
                    continue


                if newurl not in list(map(lambda r : r[0], seens)) and\
                    newurl not in list(map(lambda r : r[0], URLs)):
                    URLs.append((newurl, seed[1] +1))




KeyboardInterrupt: 

In [177]:
urlparse(url).query

'q=%EC%B9%B4%EB%A6%AC%EB%82%98&oq=%EC%B9%B4%EB%A6%AC%EB%82%98&aqs=chrome.0.69i59j69i61l2.929j0j4&sourceid=chrome&ie=UTF-8'